**Full Fine-Tuning Code Example**
---



***1. Imports***

In [ ]:
!pip install transformers

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    GPT2Tokenizer,
    GPT2LMHeadModel,
    get_cosine_schedule_with_warmup
)

from torch.optim import AdamW

***2. Custom Medical Dataset***

In [ ]:
class MedicalQADataset(Dataset):
    def __init__(self, data, tokenizer, max_len=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, idx):
        item = self.data[idx]
        prompt = f"Q: {item['instruction']}\nA:"
        answer = item["output"]

        text = prompt + " " + answer

        enc = self.tokenizer(
            text,
            max_length=self.max_len,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )
        enc = {k: v.squeeze(0) for k, v in enc.items()}
        return enc

    def __len__(self):
        return len(self.data)

***3. Load Tokenizer & Model***

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

model = GPT2LMHeadModel.from_pretrained("gpt2")

# Full fine-tuning:
for param in model.parameters():
    param.requires_grad = True

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


 ***4. Prepare Dataset & Loader***

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json
dataset_path = "/content/drive/MyDrive/Colab Notebooks/LLM_demo/datasets/medical_dataset.jsonl"
with open(dataset_path, "r") as f:
    dataset_raw = [json.loads(l) for l in f]

dataset = MedicalQADataset(dataset_raw, tokenizer)
loader = DataLoader(dataset, batch_size=4, shuffle=True)

***5. Optimizer & Scheduler***

In [ ]:
epochs = 100
optimizer = AdamW(model.parameters(), lr=5e-5, weight_decay=0.01)

num_training_steps = len(loader) * epochs
num_warmup_steps = int(0.1 * num_training_steps)

scheduler = get_cosine_schedule_with_warmup(
    optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps
)

# Mixed precision
scaler = torch.cuda.amp.GradScaler()

model.cuda()

/tmp/ipython-input-1821959825.py:14: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

***6. Training Loop***

In [ ]:
import os
os.makedirs("checkpoints", exist_ok=True)

In [ ]:
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in loader:
        batch = {k: v.cuda() for k, v in batch.items()}

        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            outputs = model(
                input_ids=batch["input_ids"],
                attention_mask=batch["attention_mask"],
                labels=batch["input_ids"]
            )
            loss = outputs.loss

        scaler.scale(loss).backward()

        # Gradient clipping (important for LLMs)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs} | Loss: {total_loss / len(loader):.4f}")

    # -------------------------
    # 6. Save checkpoint
    # -------------------------
    torch.save(model.state_dict(), f"checkpoints/epoch_{epoch+1}.pth")

/tmp/ipython-input-3787497112.py:10: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Epoch 1/100 | Loss: 9.3752
Epoch 2/100 | Loss: 5.3520
Epoch 3/100 | Loss: 0.7575
Epoch 4/100 | Loss: 0.1711
Epoch 5/100 | Loss: 0.1217
Epoch 6/100 | Loss: 0.0969
Epoch 7/100 | Loss: 0.0814
Epoch 8/100 | Loss: 0.0694
Epoch 9/100 | Loss: 0.0564
Epoch 10/100 | Loss: 0.0453
Epoch 11/100 | Loss: 0.0388
Epoch 12/100 | Loss: 0.0338
Epoch 13/100 | Loss: 0.0273
Epoch 14/100 | Loss: 0.0262
Epoch 15/100 | Loss: 0.0222
Epoch 16/100 | Loss: 0.0213
Epoch 17/100 | Loss: 0.0184
Epoch 18/100 | Loss: 0.0178
Epoch 19/100 | Loss: 0.0142
Epoch 20/100 | Loss: 0.0151
Epoch 21/100 | Loss: 0.0150
Epoch 22/100 | Loss: 0.0130
Epoch 23/100 | Loss: 0.0120
Epoch 24/100 | Loss: 0.0121
Epoch 25/100 | Loss: 0.0122
Epoch 26/100 | Loss: 0.0121
Epoch 27/100 | Loss: 0.0120
Epoch 28/100 | Loss: 0.0118
Epoch 29/100 | Loss: 0.0113
Epoch 30/100 | Loss: 0.0109
Epoch 31/100 | Loss: 0.0107
Epoch 32/100 | Loss: 0.0105
Epoch 33/100 | Loss: 0.0110
Epoch 34/100 | Loss: 0.0106
Epoch 35/100 | Loss: 0.0100
Epoch 36/100 | Loss: 0.0102
E

***7. Testing Models***

In [ ]:
### 1. Load Your Fine-Tuned Model + Tokenizer
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

# Path to your saved checkpoint (modify as needed)
checkpoint_path = "/content/checkpoints/epoch_99.pth"

# Load tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# Load pre-trained GPT2 model
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Load your fine-tuned weights
model.load_state_dict(torch.load(checkpoint_path, map_location="cuda"))
model = model.cuda()
model.eval()

print("Model loaded successfully!")

Model loaded successfully!


In [ ]:
### 2. Write a Helper Function to Ask Questions
def ask_medical_question(question, max_new_tokens=100):
    prompt = f"Q: {question}\nA:"

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    return answer


In [ ]:
print(ask_medical_question("What are symptoms of anemia?"))
print(ask_medical_question("How do doctors treat viral infections?"))
print(ask_medical_question("What causes chest pain?"))
print(ask_medical_question("How is dehydration treated?"))
print(ask_medical_question("What are signs of kidney failure?"))
print(ask_medical_question("Explain how fever works."))

Q: What are symptoms of anemia?
A: Fatigue, pale skin, dizziness, shortness of breath, cold hands and feet.
Q: How do doctors treat viral infections?
A: Viruses, bacteria, or fungi infecting the lungs.
Q: What causes chest pain?
A: Injury, infection, autoimmune diseases like rheumatoid arthritis, or overuse.
Q: How is dehydration treated?
A: Rest, fluids, fever reducers, and oxygen therapy when needed.
Q: What are signs of kidney failure?
A: Fatigue, pale skin, dizziness, shortness of breath, cold hands and feet.
Q: Explain how fever works.
A: It causes headache, fatigue, dizziness, dry mouth, and can lead to low blood pressure or heat stroke.
